<a href="https://colab.research.google.com/github/rlagosb/GastricCancerIncidence/blob/main/1_Participants_and_Trajectories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook presenta un análisis de la incidencia de cáncer gástrico, las características demográficas de la población de estudio y objetivo, las trayectorias de los pacientes y la sobrevida intrahospitalaria. El objetivo es proporcionar información detallada para investigadores interesados en la epidemiología del cáncer gástrico y sus resultados.

El contenido incluye:

*   **Carga y preparación de datos:** Se cargan y procesan datos de poblaciones, egresos hospitalarios y defunciones relacionados con el cáncer digestivo.
*   **Caracterización de participantes (Tabla 1):** Se presenta una tabla descriptiva de la población de estudio y objetivo, estratificada por sexo, rangos de edad, ruralidad y tipo de aseguramiento (Fonasa/Isapre).
*   **Trayectorias de Cáncer Gástrico:** Se analizan las secuencias de eventos (egreso, defunción) de los pacientes, incluyendo un diagrama de Sankey para visualizar los flujos de transición.
*   **Trayectorias por provincia:** Se examinan las defunciones sin hospitalización y la sobrevida a 5 años por provincia y quinquenio, para tener una perspectiva regional de los resultados.

# Setup 💾

In [1]:
import pandas as pd
import plotly.graph_objects as go

path_cubo = 'https://raw.githubusercontent.com/rlagosb/GastricCancerIncidence/main/Data/'

# cargar cubo poblaciones
cubo = pd.read_parquet(path_cubo + 'CUBO_CANCER_DIGESTIVO.parquet')
cubo = cubo[cubo.Provincia!=122] #excluir provincia Antártica
print(cubo.info(), end='\n\n')

# cargar provincias
provincias = (cubo[['Region','Provincia','Nombre Region','Nombre Provincia']].
              drop_duplicates())

# Cargar egresos-defunciones desagregados
egresosDefs = pd.read_parquet(path_cubo + 'EGRESOS_DEFUNCIONES_DESAGREGADOS.parquet')
egresosDefs = egresosDefs[egresosDefs.Provincia!=122] #excluir provincia Antártica
print(egresosDefs.info())

<class 'pandas.core.frame.DataFrame'>
Index: 22176 entries, 0 to 22967
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Provincia             22176 non-null  int64  
 1   Nombre Provincia      22176 non-null  object 
 2   Nombre Region         22176 non-null  object 
 3   Region                22176 non-null  int64  
 4   Macrorregion          22176 non-null  object 
 5   Año                   22176 non-null  int64  
 6   Codigo                22176 non-null  int64  
 7   Sexo                  22176 non-null  object 
 8   RangoEdad10           22176 non-null  object 
 9   RangoEdad4080         22176 non-null  object 
 10  MedianaRangoEdad10    22176 non-null  float64
 11  MedianaRangoEdad4080  22176 non-null  float64
 12  Poblacion             22176 non-null  int64  
 13  PoblacionRural        22176 non-null  int64  
 14  A                     22176 non-null  int64  
 15  B                     22

# 💑 Participantes
Años-persona de población de estudio y objetivo

In [ ]:
def get_participantes(RangoEdad, categoria='C16', periodo_objetivo=range(2003,2025)):

  # Filtrar tipo de cáncer
  df = cubo[cubo.Categoria==categoria].copy()

  # Agrupar edad por rango
  metricas = ['Poblacion','PoblacionRural','Fonasa','A','B','C','D','Isapre']
  df = df.groupby(['Año','Sexo',RangoEdad,'RPC','Provincia'],observed=True)[metricas].sum().reset_index()

  # Generar población de estudio y objetivo
  df1 = df[df.RPC>0].copy()
  df2 = df[df.Año.isin(periodo_objetivo)].copy()
  df1['Grupo'] = 'Estudio'
  df2['Grupo'] = 'Objetivo'
  df = pd.concat([df1,df2])

  # Contar Años
  agg_dict = {metric: 'sum' for metric in metricas}
  agg_dict['Año'] = 'count'
  df = df.groupby(['Sexo', RangoEdad,'RPC','Provincia','Grupo'], observed=True).agg(agg_dict).reset_index()

  return df

poblaciones = get_participantes('RangoEdad4080')
poblaciones

,Sexo,RangoEdad4080,RPC,Provincia,Grupo,Poblacion,PoblacionRural,Fonasa,A,B,C,D,Isapre,Año
0,Hombre,"[0, 40)",0,11,Objetivo,2209190,34920,1453505,405604,340120,302172,405609,542627,22
1,Hombre,"[0, 40)",0,14,Objetivo,175192,81561,122398,50237,23944,18164,30277,13795,22
2,Hombre,"[0, 40)",0,21,Objetivo,1287422,28215,773474,144760,190287,154054,284451,355918,9
3,Hombre,"[0, 40)",0,22,Objetivo,567792,31083,377674,55864,88600,73766,159624,155859,9
4,Hombre,"[0, 40)",0,23,Objetivo,92156,4668,43295,15223,7825,6584,13663,17205,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,Mujer,"[80, 120)",6,72,Objetivo,5087,1336,4332,27,3955,149,201,51,3
944,Mujer,"[80, 120)",6,73,Estudio,14774,3929,7950,53,11740,517,582,320,3
945,Mujer,"[80, 120)",6,73,Objetivo,14774,3929,7950,53,11740,517,582,320,3
946,Mujer,"[80, 120)",6,74,Estudio,16191,5496,13381,155,13499,564,583,277,3


## Tabla 1

Según recomendaciones de Hayes-Larson, Eleanor, Katrina L. Kezios, Stephen J. Mooney, y Gina Lovasi. «Who Is in This Study, Anyway? Guidelines for a Useful Table 1». Journal of Clinical Epidemiology 114 (octubre de 2019): 125-32. https://doi.org/10.1016/j.jclinepi.2019.06.011.


In [ ]:
def subtabla(variable, RangoEdad, df) -> pd.DataFrame:

  # sumar total población

  match variable:
    case 'Total':
      df = df.groupby(['Grupo'],observed=True)['Poblacion'].sum().reset_index()
      df['Total'] = df['Grupo'] + ' N=' + df['Poblacion'].astype(str)
      return df.Total.to_list()

    case 'Sexo' | 'RangoEdad4080':
      df = df.pivot_table(index=variable, values='Poblacion',columns='Grupo',aggfunc='sum',margins=True)
      # transform to percentages
      df['Estudio'] = df['Estudio']/df.loc['All','Estudio']
      df['Objetivo'] = df['Objetivo']/df.loc['All','Objetivo']
      return df.drop('All').drop(columns='All').reset_index(names=variable)

    case 'Ruralidad':
      df = df.groupby(['Grupo'], observed=True)[['PoblacionRural','Poblacion']].sum().reset_index()
      df['Ruralidad'] = df['PoblacionRural']/df['Poblacion']
      return df[['Grupo','Ruralidad']].set_index('Grupo').T

    case 'Beneficiarios':
      tramos = ['A','B','C','D','Isapre']
      df = df.groupby(['Grupo'], observed=True)[['Poblacion'] + tramos].sum().reset_index()
      for col in tramos:
        df[col] = df[col]/df['Poblacion']
      return df.set_index('Grupo').T.drop('Poblacion').reset_index(names='Beneficiarios')


def get_tabla1(RangoEdad, df):

  # Agrupar tablas
  tabla = pd.DataFrame()
  for variable in ['Sexo',RangoEdad,'Ruralidad','Beneficiarios']:
    tabla_var = subtabla(variable, RangoEdad, df)
    tabla_var['Variable'] = variable
    tabla_var.rename(columns={variable:'Valor'},inplace=True)
    tabla = pd.concat([tabla, tabla_var])

  # Formatear índice y columnas
  tabla.set_index(['Variable','Valor'], inplace=True)
  tabla.columns = subtabla('Total', 'RangoEdad4080', poblaciones)

  return (tabla*100).round(1)


tabla1 = get_tabla1('RangoEdad4080', poblaciones)
tabla1


Estudio N=29348519  Objetivo N=394057268
Variable      Valor                                              
Sexo          Hombre                   49.6                  49.3
              Mujer                    50.4                  50.7
RangoEdad4080 [0, 40)                  61.9                  59.8
              [40, 50)                 13.9                  13.7
              [50, 60)                 11.1                  11.5
              [60, 70)                  7.1                   8.0
              [70, 80)                  4.1                   4.7
              [80, 120)                 2.0                   2.4
Ruralidad     NaN                      15.5                  12.3
Beneficiarios A                        18.7                  19.6
              B                        20.4                  26.4
              C                        10.1                  12.5
              D                        12.1                  16.4
              Isapre                   17.3                  17.9

# Trayectorias Cáncer Gástrico

In [2]:
#@title Cargar Eventos

# Eventos sin id Persona o Provincia
egresosDefs['SinIdPersona'] = (egresosDefs.idPersona.isna())
egresosDefs['SinProvincia'] = (egresosDefs.Provincia==999)

print(egresosDefs.reset_index().pivot_table(index='Categoria',
                                      columns=['Trayectoria','SinProvincia','SinIdPersona'],
                                      values='index',
                                      aggfunc='count',
                                      margins=True))

# Filtrar eventos sin id Persona o Provincia
egresosDefs = egresosDefs[lambda x: ~x.SinProvincia & ~x.SinIdPersona]

Trayectoria  Defuncion  Egreso                      All
SinProvincia     False   False        True             
SinIdPersona     False   False  True False True        
Categoria                                              
C16              69860  107252  2343   277    9  179741
All              69860  107252  2343   277    9  179741


In [3]:
#@title Cargar Trayectorias

def cargar_trayectorias_personas(df):
  # Concatenar eventos CG de una persona
  def serie_trayectorias(serie):
    serie = serie.astype(str) + '>'
    x = serie.cumsum()
    return x

  trayectorias = (df.groupby(['idPersona'])['Trayectoria'].
                  apply(serie_trayectorias).str[:-1]. # agrupar eventos de cada persona
                  value_counts(dropna=False))         # contar personas con misma trayectoria
  return trayectorias.reset_index()

trayectorias = cargar_trayectorias_personas(egresosDefs.copy())

print('Top 5 trayectorias\n:', trayectorias.head(5), end='\n\n')

print(trayectorias['count'].sum(), 'personas con trayectorias de CG,\n',
      trayectorias[lambda x: x.Trayectoria.str.contains('Defuncion')]['count'].sum()*100/trayectorias['count'].sum(), '% con defunción\n',
      trayectorias[lambda x: x.Trayectoria.str.contains('Egreso')]['count'].sum()*100/trayectorias['count'].sum(), '% con egreso\n',
      trayectorias[lambda x: x.Trayectoria.str.contains('Defuncion') & ~x.Trayectoria.str.contains('Egreso')]['count'].sum()*100/trayectorias['count'].sum(), '% con defunción sin egreso')

Top 5 trayectorias
:             Trayectoria  count
0                Egreso  62862
1             Defuncion  33223
2      Egreso>Defuncion  23058
3         Egreso>Egreso  20217
4  Egreso>Egreso>Egreso   8359

177112 personas con trayectorias de CG,
 39.46542300916934 % con defunción
 81.24068386105967 % con egreso
 18.75931613894033 % con defunción sin egreso


In [ ]:
#@title Figura 1

def diagrama_sankey(trayectorias):

  # Parametrizar transiciones
  for col in ['Sano_Egreso','Sano_Defuncion','Egreso_Vivo','Egreso_Defuncion']:
    trayectorias[col] = False
  egresoInicio = (trayectorias.Trayectoria.str[:6]=='Egreso')
  conDefuncion = (trayectorias.Trayectoria.str.contains('Defuncion'))
  # Sano>Egreso
  trayectorias.loc[egresoInicio,'Sano_Egreso'] = True
  # Sano>Defuncion
  trayectorias.loc[trayectorias.Trayectoria.str[:9]=='Defuncion','Sano_Defuncion'] = True
  # Egreso>Vivo
  trayectorias.loc[egresoInicio & ~conDefuncion,'Egreso_Vivo'] = True
  # Egreso>Defuncion
  trayectorias.loc[egresoInicio & conDefuncion,'Egreso_Defuncion'] = True

  arcos = (trayectorias.melt(id_vars=['Trayectoria','count'],
                           value_vars=['Sano_Egreso','Sano_Defuncion','Egreso_Vivo','Egreso_Defuncion'],
                           var_name='Arco',value_name='Valor')[lambda x: x.Valor].  # dejar sólo transiciones activas
           groupby(['Arco','Valor'])['count'].sum().reset_index())                  # agrupar transiciones
  # separar origen/destino
  arcos[['Source','Target']] = (arcos.Arco.str.replace('Egreso','Hospitalización').str. # cambiar nombre Egreso
                                split('_',expand=True))

  # Create Sankey diagram https://plotly.com/python/sankey-diagram/
  nodes = pd.concat([arcos['Source'], arcos['Target']]).unique()
  nodes = pd.DataFrame({'name': nodes})
  nodes['id'] = nodes.index
  arcos = arcos.merge(nodes.rename(columns={'name':'Source'}), on='Source', how='left')
  arcos = arcos.merge(nodes.rename(columns={'name':'Target'}), on='Target', how='left', suffixes=('_source', '_target'))

  go.Figure(data=[go.Sankey(
      node=dict(pad=15, thickness=20,
          line=dict(color="black", width=0.5),
          label=nodes['name'],),
      link=dict(
          source=arcos['id_source'],
          target=arcos['id_target'],
          value=arcos['count']))]).show()


diagrama_sankey(trayectorias.copy())

# Trayectorias Provincias

In [5]:
def trayectorias_persona(df):
  df = df[df.idPersona.notna()]

  fa = df[df.Trayectoria=='Defuncion'][['idPersona','Año']].rename(columns={'Año':'AñoDefuncion'})
  eg = df[df.Trayectoria=='Egreso'][['idPersona','Año']].rename(columns={'Año':'AñoEgreso'})

  # Obtener año de egreso y hospitalización
  df = (df.groupby(['idPersona']).agg({'Provincia':'first'}).reset_index().
        merge(eg, on='idPersona', how='outer').
        merge(fa, on='idPersona', how='outer').
        groupby(['idPersona','Provincia']).agg({'AñoEgreso':'min','AñoDefuncion':'max'}).reset_index())

  # Métricas sobrevida
  df['Delta'] = df.AñoDefuncion - df.AñoEgreso
  df.loc[df.AñoDefuncion.isna(), 'sobrevida5'] = True
  df.loc[df.Delta.notna(), 'sobrevida5'] = df[df.Delta.notna()].Delta.apply(lambda x: x>=5)

  # get minimum between AñoEgreso and AñoDefuncion
  df['Quinquenio'] = df[['AñoEgreso','AñoDefuncion']].min(axis=1).apply(lambda x: (x//5)*5)

  df['Defuncion'] = df.AñoDefuncion.notna()
  df['Egreso'] = df.AñoEgreso.notna()
  df['DefuncionSinEgreso'] = df.Defuncion & ~df.Egreso

  df = df.groupby(['Provincia','Quinquenio']).agg({'Delta':'mean','sobrevida5':'sum','Defuncion':'sum','Egreso':'sum','DefuncionSinEgreso':'sum','idPersona':'count'}).reset_index()

  return df

trayectoriasProv = trayectorias_persona(egresosDefs.copy())

## Defunciones sin hospitalización (Material Sup 3)

In [ ]:
# Defunción sin hospitalización por provincia y quinquenio

print('Porcentaje de defunciones sin hospitalización:\n',
      trayectoriasProv.DefuncionSinEgreso.sum()*100/trayectoriasProv.idPersona.sum())

trayectoriasProv['PorcDefSinEgreso'] = trayectoriasProv.DefuncionSinEgreso*100/trayectoriasProv.idPersona

trayectoriasProv[lambda x: x.Quinquenio>2000].pivot(columns='Quinquenio',index='Provincia',values='PorcDefSinEgreso').\
merge(provincias, on='Provincia').\
set_index(['Region','Nombre Region','Nombre Provincia']).drop(columns='Provincia').round(1)

Porcentaje de defunciones sin hospitalización:
 34.58227057415023


2005.0  \
Region Nombre Region                             Nombre Provincia           
1      Tarapacá                                  Iquique             67.0   
                                                 Tamarugal           77.8   
2      Antofagasta                               Antofagasta         31.9   
                                                 El Loa              29.6   
                                                 Tocopilla           16.0   
3      Atacama                                   Copiapó             34.6   
                                                 Chañaral            23.3   
                                                 Huasco              28.2   
4      Coquimbo                                  Elqui               40.3   
                                                 Choapa              52.2   
                                                 Limarí              39.2   
5      Valparaíso                                Valparaíso          25.4   
                                                 Isla de Pascua      25.0   
                                                 Los Andes           33.3   
                                                 Petorca             37.8   
                                                 Quillota            34.7   
                                                 San Antonio         38.5   
                                                 San Felipe          14.7   
                                                 Marga Marga         37.6   
6      Libertador General Bernardo O'Higgins     Cachapoal           34.7   
                                                 Cardenal Caro       45.8   
                                                 Colchagua           38.1   
7      Maule                                     Talca               44.0   
                                                 Cauquenes           55.1   
                                                 Curicó              40.1   
                                                 Linares             41.4   
8      Biobío                                    Concepción          28.8   
                                                 Arauco              33.4   
                                                 Bíobío              33.1   
9      La Araucanía                              Cautín              37.9   
                                                 Malleco             28.9   
10     Los Lagos                                 Llanquihue          29.0   
                                                 Chiloé              45.1   
                                                 Osorno              37.0   
                                                 Palena              30.6   
11     Aysén del General Carlos Ibáñez del Campo Coyhaique           38.1   
                                                 Aysén               43.9   
                                                 Capitán Prat        60.0   
                                                 General Carrera     38.5   
12     Magallanes y de la Antártica Chilena      Magallanes          30.6   
                                                 Tierra del Fuego    20.0   
                                                 Última Esperanza    52.0   
13     Metropolitana de Santiago                 Santiago            34.4   
                                                 Cordillera          24.4   
                                                 Chacabuco           39.6   
                                                 Maipo               36.7   
                                                 Melipilla           51.5   
                                                 Talagante           53.1   
14     Los Ríos                                  Valdivia            26.4   
                                                 Ranco               24.0   
15     Arica y Parinacota                        Arica         

## Sobrevida Intrahospitalaria (Material Sup 4)


In [8]:
# add colum percentage sobrevida5

# Agregar total nacional
sobre = pd.concat([trayectoriasProv.copy(),
                  trayectoriasProv.groupby(['Quinquenio'])[['sobrevida5','idPersona']].sum().reset_index()])
# Excluir último quinquenio
sobre = sobre[sobre.Quinquenio<2020]

# Calcular sobrevida 5 años
sobre['PorcSobrevida5'] = (sobre.sobrevida5*100.0/sobre.idPersona).round(1)

print('Sobrevida a 5 años nacional:\n',   sobre[sobre.Provincia.isna()].sobrevida5.sum()/sobre[sobre.Provincia.isna()].idPersona.sum())


Sobrevida a 5 años nacional:
 0.26566419384106316


In [ ]:
sobre = sobre.pivot(index='Provincia', columns='Quinquenio', values='PorcSobrevida5').\
merge(provincias, on='Provincia', how='left').\
drop(columns='Provincia').set_index(['Region','Nombre Region','Nombre Provincia'])

for col in sobre.columns:
  sobre[col] = sobre[col].astype(float).round(1)

sobre

2000.0  \
Region Nombre Region                             Nombre Provincia           
NaN    NaN                                       NaN                 26.7   
1.0    Tarapacá                                  Iquique             14.1   
                                                 Tamarugal           33.3   
2.0    Antofagasta                               Antofagasta         23.6   
                                                 El Loa              37.8   
                                                 Tocopilla           15.8   
3.0    Atacama                                   Copiapó             39.3   
                                                 Chañaral            56.2   
                                                 Huasco              13.2   
4.0    Coquimbo                                  Elqui               26.3   
                                                 Choapa              22.2   
                                                 Limarí              20.4   
5.0    Valparaíso                                Valparaíso          25.5   
                                                 Isla de Pascua       NaN   
                                                 Los Andes           32.0   
                                                 Petorca             25.8   
                                                 Quillota            27.1   
                                                 San Antonio         15.5   
                                                 San Felipe          24.7   
                                                 Marga Marga         35.2   
6.0    Libertador General Bernardo O'Higgins     Cachapoal           20.5   
                                                 Cardenal Caro       29.4   
                                                 Colchagua           17.6   
7.0    Maule                                     Talca               16.6   
                                                 Cauquenes           10.3   
                                                 Curicó              16.7   
                                                 Linares             16.9   
8.0    Biobío                                    Concepción          41.3   
                                                 Arauco              38.3   
                                                 Bíobío              23.2   
9.0    La Araucanía                              Cautín              17.4   
                                                 Malleco             21.0   
10.0   Los Lagos                                 Llanquihue          22.8   
                                                 Chiloé              14.8   
                                                 Osorno              26.6   
                                                 Palena              57.1   
11.0   Aysén del General Carlos Ibáñez del Campo Coyhaique           16.7   
                                                 Aysén               41.2   
                                                 Capitán Prat        50.0   
                                                 General Carrera      0.0   
12.0   Magallanes y de la Antártica Chilena      Magallanes          26.6   
                                                 Tierra del Fuego    12.5   
                                                 Última Esperanza    12.5   
13.0   Metropolitana de Santiago                 Santiago            28.8   
                                                 Cordillera          35.5   
                                                 Chacabuco           26.1   
                                                 Maipo               21.9   
                                                 Melipilla           23.9   
                                                 Talagante           18.8   
14.0   Los Ríos                                  Valdivia            27.2   
                                                 Ranco         